In [35]:
import sys
sys.path.append('..')

In [36]:
import numpy

In [37]:
p = numpy.array(
    [
        [11, 12, 13],
        [21, 22, 23],
        [31, 32, 33],
    ]
)
p

array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

In [38]:
import lasp.utils
p1 = lasp.utils.circshift(p, center=1-numpy.array([2, 2]))
print(p1)

[[22 23 21]
 [32 33 31]
 [12 13 11]]


In [39]:
def make_bccb(p: numpy.ndarray) -> numpy.ndarray :
    
    dim = numpy.prod(numpy.array(p.shape))
    p_cpy = numpy.copy(p)
    
    p1 = numpy.reshape(p_cpy.T, newshape=dim)
    P = numpy.zeros(shape=(dim, dim))
    P[:, 0] = p1

    for i in range(1, 3):
        for j in range(0, 3*3, 3):
            P[j:j+3, i] = numpy.roll(P[j:j+3, i-1], 1)

    for i in range(1, 3):
        P[:, 3*i: 3*i+3] = numpy.roll(P[:, 3*(i-1): 3*(i-1)+3], (3, 0), (0, 1))


    return P

In [40]:
make_bccb(p1)

array([[22., 12., 32., 21., 11., 31., 23., 13., 33.],
       [32., 22., 12., 31., 21., 11., 33., 23., 13.],
       [12., 32., 22., 11., 31., 21., 13., 33., 23.],
       [23., 13., 33., 22., 12., 32., 21., 11., 31.],
       [33., 23., 13., 32., 22., 12., 31., 21., 11.],
       [13., 33., 23., 12., 32., 22., 11., 31., 21.],
       [21., 11., 31., 23., 13., 33., 22., 12., 32.],
       [31., 21., 11., 33., 23., 13., 32., 22., 12.],
       [11., 31., 21., 13., 33., 23., 12., 32., 22.]])

In [41]:
x = numpy.array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])
numpy.roll(x, (1, 0), (0, 1))
# array([[9, 0, 1, 2, 3],
#        [4, 5, 6, 7, 8]])

array([[5, 6, 7, 8, 9],
       [0, 1, 2, 3, 4]])

In [42]:
def make_bccb(h: numpy.ndarray) -> numpy.ndarray :
    
    size_block = h.shape[0]

    center = numpy.ceil(numpy.array(h.shape) / 2).astype(int)
    shift = 1-center
    h_shift = lasp.utils.circshift(h, shift)

    dim = size_block**2

    col = numpy.reshape(h_shift.T, newshape=dim)
    
    bccb = numpy.zeros(shape=(dim, dim))
    bccb[:, 0] = col

    for i in range(1, h.shape[0]):
        for j in range(0, size_block**2, size_block):
            bccb[j:j+size_block, i] = numpy.roll(bccb[j:j+size_block, i-1], 1)

    for i in range(1, 3):
        bccb[:, size_block*i: size_block*i+size_block] = \
            numpy.roll(
                bccb[:, size_block*(i-1): size_block*(i-1)+size_block], 
                (size_block, 0), 
                (0, 1)
            )


    return bccb

In [43]:
h = numpy.array(
    [
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ]
)

x_mat = numpy.reshape(numpy.arange(0, 9), (3, 3))
x_vec = numpy.reshape(x_mat, newshape=9, order='F')
print(x_mat)
print(x_vec)


[[0 1 2]
 [3 4 5]
 [6 7 8]]
[0 3 6 1 4 7 2 5 8]


In [44]:
H = make_bccb(h)
H @ x_vec

array([210., 129., 210., 201., 120., 201., 210., 129., 210.])

In [47]:
import scipy.fft

size_block = h.shape[0]
center = numpy.ceil(numpy.array(h.shape) / 2).astype(int)
shift = 1-center
h_shift = lasp.utils.circshift(h, shift)

s = scipy.fft.fft2(h_shift)
b = scipy.fft.ifft2(s * scipy.fft.fft(x_mat))
b = numpy.real(b)
print(b)


[[ 16. +15.58845727j  13. +15.58845727j  16. +15.58845727j]
 [ 65.5 -7.79422863j  62.5 -7.79422863j  65.5 -7.79422863j]
 [-33.5 -7.79422863j -36.5 -7.79422863j -33.5 -7.79422863j]]


In [ ]:
import scipy.fft

lamda = scipy.fft.fft(H[:, 0])
lamda

array([45.         -0.j        , -2.81907786 -3.35964617j,
        2.29813333 +0.40522291j,  0.        -15.58845727j,
        0.52094453 -1.43128334j,  0.52094453 +1.43128334j,
        0.        +15.58845727j,  2.29813333 -0.40522291j,
       -2.81907786 +3.35964617j])

In [ ]:
size_block = h.shape[0]

center = numpy.ceil(numpy.array(h.shape) / 2).astype(int)
shift = 1-center
h_shift = lasp.utils.circshift(h, shift)
scipy.fft.fft2(h_shift)

array([[45. -0.j        ,  0. -5.19615242j,  0. +5.19615242j],
       [ 0.-15.58845727j,  0. +0.j        ,  0. -0.j        ],
       [ 0.+15.58845727j,  0. +0.j        ,  0. -0.j        ]])

In [ ]:
x_vec = numpy.arange(0, 9)
x_mat = numpy.reshape(x_vec, (3, 3))

In [ ]:
print(x_vec)
print(x_mat)

[0 1 2 3 4 5 6 7 8]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [ ]:
scipy.fft.fft2(x_mat)

array([[ 36. -0.j        ,  -4.5+2.59807621j,  -4.5-2.59807621j],
       [-13.5+7.79422863j,   0. +0.j        ,   0. -0.j        ],
       [-13.5-7.79422863j,   0. +0.j        ,   0. -0.j        ]])

In [ ]:
s